# Spaceship Lightgbm Submission

In [ ]:
import pandas as pd
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Bring the Data In

In [ ]:
X_df = pd.read_csv('spaceship_train_X_v2.csv')
y_df = pd.read_csv('spaceship_train_y.csv')
X_submission = pd.read_csv('spaceship_test_X_v2.csv')

y_df.drop('Unnamed: 0', axis=1, inplace=True)
X_df.head()

,PassengerId,CryoSleep,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Expenditure,NoSpending,...,CabinSide_Z,CabinDeck_A,CabinDeck_B,CabinDeck_C,CabinDeck_D,CabinDeck_E,CabinDeck_F,CabinDeck_G,CabinDeck_T,AgeEncoded
0,0001_01,0,0,-0.337530,-0.283865,-0.287383,-0.273826,-0.265831,-0.518357,1,...,0,0,1,0,0,0,0,0,0,3.0
1,0002_01,0,0,-0.172455,-0.278226,-0.245562,0.213965,-0.227033,-0.256582,0,...,0,0,0,0,0,0,1,0,0,2.0
2,0003_01,0,1,-0.272409,1.956643,-0.287383,5.692512,-0.222625,3.174596,0,...,0,1,0,0,0,0,0,0,0,4.0
3,0003_02,0,0,-0.337530,0.519986,0.333240,2.684020,-0.095651,1.322607,0,...,0,1,0,0,0,0,0,0,0,2.0
4,0004_01,0,0,0.121349,-0.240007,-0.034784,0.228181,-0.264067,-0.130318,0,...,0,0,0,0,0,0,1,0,0,1.0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)
# Mengubah target menjadi 1D array
y_train = y_train['Transported'].values
y_test = y_test['Transported'].values

# Base estimator
base_estimators = [
    ('svc', SVC(probability=True)),
    ('dt', DecisionTreeClassifier()),
    ('knn', KNeighborsClassifier())
]

# Membuat pipeline
stacking_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Preprocessing: Normalisasi data
    ('stacking', StackingClassifier(estimators=base_estimators, final_estimator=LogisticRegression()))
])

param_grid = {
    'stacking__svc__C': [10],
    'stacking__dt__max_depth': [20],
    'stacking__knn__n_neighbors': [10],
    'stacking__final_estimator__C': [10]
}

# Model Training

In [ ]:
# Membuat GridSearchCV
grid_search = GridSearchCV(stacking_pipeline, param_grid, cv=5, scoring='accuracy', verbose=10)
grid_search.fit(X_train, y_train)

#Evaluasi model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Best parameters:", grid_search.best_params_)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START stacking__dt__max_depth=20, stacking__final_estimator__C=10, stacking__knn__n_neighbors=10, stacking__svc__C=10
[CV 1/5; 1/1] END stacking__dt__max_depth=20, stacking__final_estimator__C=10, stacking__knn__n_neighbors=10, stacking__svc__C=10;, score=0.792 total time=  55.0s
[CV 2/5; 1/1] START stacking__dt__max_depth=20, stacking__final_estimator__C=10, stacking__knn__n_neighbors=10, stacking__svc__C=10
[CV 2/5; 1/1] END stacking__dt__max_depth=20, stacking__final_estimator__C=10, stacking__knn__n_neighbors=10, stacking__svc__C=10;, score=0.793 total time=  54.8s
[CV 3/5; 1/1] START stacking__dt__max_depth=20, stacking__final_estimator__C=10, stacking__knn__n_neighbors=10, stacking__svc__C=10
[CV 3/5; 1/1] END stacking__dt__max_depth=20, stacking__final_estimator__C=10, stacking__knn__n_neighbors=10, stacking__svc__C=10;, score=0.791 total time=  53.9s
[CV 4/5; 1/1] START stacking__dt__max_depth=20, stacking

In [ ]:
# Evaluasi pada validation set
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.81      0.80       861
           1       0.81      0.79      0.80       878

    accuracy                           0.80      1739
   macro avg       0.80      0.80      0.80      1739
weighted avg       0.80      0.80      0.80      1739

Confusion Matrix:
[[694 167]
 [187 691]]


In [ ]:
best_gak = grid_search.best_estimator_
y_pred_light = best_gak.predict(X_test)
print("Best parameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_light))
print("Classification Report:\n", classification_report(y_test, y_pred_light))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_light))

Best parameters: {'stacking__dt__max_depth': 20, 'stacking__final_estimator__C': 10, 'stacking__knn__n_neighbors': 10, 'stacking__svc__C': 10}
Accuracy: 0.7964347326049454
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.81      0.80       861
           1       0.81      0.79      0.80       878

    accuracy                           0.80      1739
   macro avg       0.80      0.80      0.80      1739
weighted avg       0.80      0.80      0.80      1739

Confusion Matrix:
 [[694 167]
 [187 691]]


# Submission Prediction

In [ ]:
# For training, we use ALL data from spaceship_train_X_v2.csv and spaceship_train_y.csv
grid_search.fit(X_df, y_df['Transported'].values)
# Prediksi data submission
y_prediction = grid_search.predict(X_submission)
print(y_prediction)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START stacking__dt__max_depth=20, stacking__final_estimator__C=10, stacking__knn__n_neighbors=10, stacking__svc__C=10
[CV 1/5; 1/1] END stacking__dt__max_depth=20, stacking__final_estimator__C=10, stacking__knn__n_neighbors=10, stacking__svc__C=10;, score=0.736 total time= 1.3min
[CV 2/5; 1/1] START stacking__dt__max_depth=20, stacking__final_estimator__C=10, stacking__knn__n_neighbors=10, stacking__svc__C=10
[CV 2/5; 1/1] END stacking__dt__max_depth=20, stacking__final_estimator__C=10, stacking__knn__n_neighbors=10, stacking__svc__C=10;, score=0.761 total time= 1.4min
[CV 3/5; 1/1] START stacking__dt__max_depth=20, stacking__final_estimator__C=10, stacking__knn__n_neighbors=10, stacking__svc__C=10
[CV 3/5; 1/1] END stacking__dt__max_depth=20, stacking__final_estimator__C=10, stacking__knn__n_neighbors=10, stacking__svc__C=10;, score=0.755 total time= 1.4min
[CV 4/5; 1/1] START stacking__dt__max_depth=20, stacking

In [ ]:
submission_dict = {'PassengerId':X_submission['PassengerId'], 'Transported':y_prediction.astype('bool')}
submission_dict = pd.DataFrame(submission_dict)
submission_dict

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


## Export CSV

In [ ]:
submission_dict.to_csv('spaceship_gabungg.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2a79941c-6614-47fe-9427-0e9f23998893' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>